In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
dataset=pd.read_csv("spam.txt",delimiter='\t',quoting=3)
dataset.head()

,Category,Message
0,ham,"""Go until jurong point, crazy.. Available only..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"""Nah I don't think he goes to usf, he lives ar..."


In [3]:
dataset.loc[dataset['Category']=='spam','Category',]=1
dataset.loc[dataset['Category']=='ham','Category',]=0

In [4]:
dataset.shape

(5574, 2)

In [5]:
dataset.describe()

,Category,Message
count,5574,5574
unique,2,5159
top,0,"""Sorry, I'll call later"""
freq,4827,30


In [5]:
import re
import nltk

In [6]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [7]:
ps=PorterStemmer()
corpus=[]

In [8]:
for i in range(0,len(dataset)):
    review=re.sub('[^a-zA-Z]',' ',dataset['Message'][i])
    review=review.lower()
    review=review.split( )
    
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [9]:
corpus[0]

'go jurong point crazi avail bugi n great world la e buffet cine got amor wat'

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000)
x=cv.fit_transform(corpus).toarray()
print(x[0])

[0 0 0 ... 0 0 0]


In [11]:
x.shape

(5574, 5000)

In [12]:
y=dataset['Category']
y.head()

0    0
1    0
2    1
3    0
4    0
Name: Category, dtype: object

In [13]:
y=y.astype('int')

In [14]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test =train_test_split(x,y,test_size=0.2,random_state=0)

In [15]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(x_train)
X_test=sc.fit_transform(x_test)

In [16]:
X_train

array([[-0.01497718, -0.02594705, -0.01497718, ...,  0.        ,
         0.        , -0.01497718],
       [-0.01497718, -0.02594705, -0.01497718, ...,  0.        ,
         0.        , -0.01497718],
       [-0.01497718, -0.02594705, -0.01497718, ...,  0.        ,
         0.        , -0.01497718],
       ...,
       [-0.01497718, -0.02594705, -0.01497718, ...,  0.        ,
         0.        , -0.01497718],
       [-0.01497718, -0.02594705, -0.01497718, ...,  0.        ,
         0.        , -0.01497718],
       [-0.01497718, -0.02594705, -0.01497718, ...,  0.        ,
         0.        , -0.01497718]])

In [17]:
y_train

2523    0
253     0
1114    0
3741    0
3097    0
       ..
4931    1
3264    0
1653    1
2607    0
2732    0
Name: Category, Length: 4459, dtype: int32

In [18]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [19]:
classifier=Sequential([
    keras.layers.Dense(1,input_shape=(5000,),activation='sigmoid'),
    #keras.layers.Dense(1,activation='sigmoid')
])

In [20]:
classifier.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [21]:
classifier.fit(X_train,y_train,epochs=100)

Epoch 1/100
140/140 [==============================] - 1s 2ms/step - loss: 1.8097 - accuracy: 0.6688
Epoch 2/100
140/140 [==============================] - 0s 2ms/step - loss: 1.0271 - accuracy: 0.8280
Epoch 3/100
140/140 [==============================] - 0s 2ms/step - loss: 0.6419 - accuracy: 0.8888
Epoch 4/100
140/140 [==============================] - 0s 2ms/step - loss: 0.3915 - accuracy: 0.9341
Epoch 5/100
140/140 [==============================] - 0s 2ms/step - loss: 0.2280 - accuracy: 0.9590
Epoch 6/100
140/140 [==============================] - 0s 2ms/step - loss: 0.1312 - accuracy: 0.9771
Epoch 7/100
140/140 [==============================] - 0s 2ms/step - loss: 0.0785 - accuracy: 0.9847
Epoch 8/100
140/140 [==============================] - 0s 2ms/step - loss: 0.0479 - accuracy: 0.9908
Epoch 9/100
140/140 [==============================] - 0s 2ms/step - loss: 0.0329 - accuracy: 0.9935
Epoch 10/100
140/140 [==============================] - 0s 2ms/step - loss: 0.0256 - accura

In [31]:
classifier.evaluate(x_test,y_test)

35/35 [==============================] - 0s 3ms/step - loss: 0.2785 - accuracy: 0.9713


[0.2785111367702484, 0.9713004231452942]

In [32]:
y_predicted=classifier(x_test)
y_predicted=y_predicted.numpy()
y_predicted=np.array(y_predicted,dtype='i8')
#y_predicted=y_predicted.reshape(1,len(y_predicted))
print(y_predicted.dtype)
print(y_predicted.shape)
print(len(y_predicted[0]))
#y_predicted=(y_predicted>0.5)
#y_predicted[:10]

int64
(1115, 1)
1


In [26]:
for i in range(0,len(y_predicted[0])):
    if(i>0.5):
        y_predicted[0][i]=1
    else:
        y_predicted[0][i]=0
y_predicted[0][1]

1

In [26]:
y_predicted=y_predicted.ast

tf.bool

In [33]:
y_test[:10]

4681    0
4620    0
4424    0
2148    0
1003    0
348     0
4106    0
1267    0
3719    0
3233    0
Name: Category, dtype: int32

In [34]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_predicted)
cm

array([[950,   0],
       [165,   0]], dtype=int64)

In [35]:
from sklearn.metrics import accuracy_score , f1_score , precision_score , recall_score

In [36]:
y_test_pred=classifier.predict(x_test)
y_test_pred

35/35 [==============================] - 0s 3ms/step


array([[0.24137506],
       [0.3532878 ],
       [0.229818  ],
       ...,
       [0.2344017 ],
       [0.2476096 ],
       [0.40794635]], dtype=float32)

In [37]:
y_test_pred=classifier.predict(x_test)
test_accu=accuracy_score(y_test,y_predicted)
test_pre=precision_score(y_test,y_predicted)
test_re=recall_score(y_test,y_predicted)
test_f1=f1_score(y_test,y_predicted)
print("accuracy score on test data ",test_accu)
print("precision score on test data ",test_pre)
print("recall score on test data ",test_re)
print("f1 score on test data ",test_f1)

35/35 [==============================] - 0s 3ms/step
accuracy score on test data  0.852017937219731
precision score on test data  0.0
recall score on test data  0.0
f1 score on test data  0.0


c:\Users\amitb\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
